In [1]:
import dash_bootstrap_components as dbc
from dash import Dash, Input, Output, html, dcc
from dash_iconify import DashIconify
import dash_mantine_components as dmc

import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go

In [2]:
# import data
kreise_df = pd.read_csv('/home/jan/Uni/DS-Project/modules/dashboard/test/assets/kreise_df.csv')
# rename and drop columns
kreise_df = kreise_df.drop(columns=['Unnamed: 0']).rename(columns={'NAME_1': 'Bundesland', 'NAME_2': 'Region', 'NAME_3': 'Landkreis', 'ENGTYPE_3': 'Land_Stadt'})

In [3]:
%%script false --no-raise-error

def flatten_list(lst):
    flattened_list = []
    for item in lst:
        if type(item) == np.ndarray:
            flattened_list += [x for x in item.tolist()]
        else:
            flattened_list.append(item)
    return flattened_list

def parent_finder(children):
    df = kreise_df
    children_list = df[children].unique().tolist()
    parent_list = []
    parent_column = df.columns.get_loc(children) - 1
    for child in children_list:
        parent_value = df.loc[df[children] == child].iloc[:, parent_column].unique()
        parent_list.append(parent_value)
    return flatten_list(parent_list)

def value_finder(typus):
    df = kreise_df
    value_list = []
    unit_list = df[typus].unique().tolist()
    for unit in unit_list:
        value_value = round(np.mean(df.loc[df[typus] == unit]['avg_Score']), 1)
        value_list.append(value_value)
    return value_list

In [4]:
%%script false --no-raise-error

# create suburst vectors
labels = kreise_df.Bundesland.unique().tolist() + kreise_df.Region.unique().tolist() + kreise_df.Landkreis.unique().tolist()
parents = [''] * len(kreise_df.Bundesland.unique()) + parent_finder('Region') + parent_finder('Landkreis')
values = value_finder('Bundesland') + value_finder('Region') + value_finder('Landkreis')

In [5]:
# create sunburst plot
Ohren_sunburst = px.sunburst(kreise_df, path=['Bundesland', 'Region', 'Landkreis'], values='avg_Score')
Ohren_sunburst = Ohren_sunburst.update_traces(
    insidetextorientation='radial',
    hovertemplate = '<b>%{label}</b> <br>Score: %{value}')

In [6]:
app = Dash(__name__, external_stylesheets = [dbc.themes.LITERA])

In [7]:
# the style arguments for the sidebar. We use position:fixed and a fixed width
SIDEBAR_STYLE = {
    "position": "fixed",
    "top": 0,
    "left": 0,
    "bottom": 0,
    "width": "26rem",
    "padding": "2rem 1rem",
    "background-color": "#f8f9fa",
}

# the styles for the main content position it to the right of the sidebar and add some padding.
CONTENT_STYLE = {
    "margin-left": "28rem",
    "margin-right": "2rem",
    "padding": "2rem 1rem",
}

In [8]:
sidebar = html.Div([
    dbc.Row([
      html.H1('Dashboard für PV-Ausbau in "Ohren"'),
      html.Hr(),
      html.P('In this research project we seek to identify the potential of photovolatic power in highway turnoffs across Germany.')
    ]),
    # Directory
    dbc.Row([
      dbc.Col([
      html.H4('Kapitel'),
        html.Ol(start=1, children=[
          html.Li([
            html.A("Einleitung", href="#einleitung"),
          ]),
          html.Li([
            html.A("Motivation", href="#motivation"),
          ]),
          html.Li([
            html.A("Aufbau", href="#aufbau"),
          ]),
          html.Li([
            html.A("Fazit", href = "#fazit"),
          ]),
          html.Li([
            html.A("Überblick Landkreise", href="#ueberblick")
          ]),
        ])])]),
    dcc.Markdown('''
        | Mitwirkende | Matrikelnummer |
        | ---         | ---:           |
        |Yvette Brody | 1234 |
        |Marvin Hoberg| 1234 |
        |Felix Schulz | 1234 |
        |Jan Besler   |5629079|
    '''),
    # Links und Erklärungen
    dbc.Row([
      dbc.Col(["GitHub Repo",
        DashIconify(icon="bi:github"),
        #href="https://github.com/felixschulz385/ds_project",
      ]),
      dbc.Col(["How it started",
        DashIconify(icon="bi:newspaper"),
        #href="https://www.badische-zeitung.de/ob-palmer-will-solaranlagen-auf-freien-stellen-neben-bundesstrassen--211487616.html",
      ]),
      html.Hr(),
      html.P('This project is being developed as part of the \
              Data Science in Business and Economics Master degree course at the University of Tübingen.')
    ])
], style=SIDEBAR_STYLE)

In [9]:
# images
import base64
with open("/home/jan/Uni/DS-Project/modules/dashboard/test/assets/economic_model.jpeg", "rb") as file:
    economic_model_img = "data:image/jpg;base64, {}".format(base64.b64encode(file.read()).decode("utf-8"))
with open("/home/jan/Uni/DS-Project/modules/dashboard/test/assets/brandenburg_driveways.jpg", "rb") as file:
    BB_ohren = "data:image/jpg;base64, {}".format(base64.b64encode(file.read()).decode("utf-8"))
# carousel pictures
with open("/home/jan/Uni/DS-Project/modules/dashboard/test/assets/lustnauer_ohren.jpg", "rb") as file:
    carousel_1 = "data:image/jpg;base64, {}".format(base64.b64encode(file.read()).decode("utf-8"))
with open("/home/jan/Uni/DS-Project/modules/dashboard/test/assets/driveways_example_6.jpg", "rb") as file:
    carousel_2 = "data:image/jpg;base64, {}".format(base64.b64encode(file.read()).decode("utf-8"))
with open("/home/jan/Uni/DS-Project/modules/dashboard/test/assets/driveways_example_7.jpg", "rb") as file:
    carousel_3 = "data:image/jpg;base64, {}".format(base64.b64encode(file.read()).decode("utf-8"))

In [10]:
# change to app.layout if running as single page app instead
content = html.Div([
    dbc.Container([
        # Überschriften & Einleitung
        dbc.Row([
            dbc.Col(html.H1("Lustnauer 'Ohren' skaliert auf ganz Deutschland", className="text-center")
                    , className="mb-5 mt-5")
        ]),
        dbc.Row([
            dbc.Col(html.H5(children='Erkennen von ungenutzten Potenzialen in ungenutzten Flächen.'
                                     )
                    , className="mb-4")
            ]),

        dbc.Row([
            dbc.Col(html.H5(children='Auf der zweiten Seite wird eine interaktive Karte dargestellt, welche die verschiedenen Ohren ranked'
                                     'Die dritte Seite präsentiert dieselben Daten in tabellarischer Form, zum erkunden und nachschlagen.')
                    , className="mb-5")
        ]),


        # Einleitung
        dbc.Row([
            html.H3('Einleitung', id="einleitung")
        ]),
        dbc.Row([
            dcc.Markdown('''
                
            ''')
        ]),
        dbc.Row([
            dbc.Carousel(
                items=[
                    {"key": "1",
                    "src": carousel_1,
                    "header": "Beispiel Ohren",
                    "caption": "",},
                    {"key": "2",
                    "src": carousel_2,
                    "header": "ausgeschnittene Flächen",
                    "caption": "",},
                    {"key": "3",
                    "src": carousel_3,
                    "header": "zusammengesetztes Bild",
                    "caption": "",},
            ], style = {
                'width': '80vh',
                'height': '80vh',
                'padding': '10vh 5vh'
                }),
            html.Hr()
        ]),
        
        # Motivation
        dbc.Row([
            html.H3('Motivation', id="motivation")
        ]),
        dbc.Row([
            dcc.Markdown('''
                Global climate change has put Germany's energy industry in a state of turmoil. 
                After years of heavy relicance on coal and gas (in 2010, 25% of the German Energy Mix 
                were sourced this way ([BDEW, 2022](https://www.bdew.de/energie/bruttostromerzeugung-seit-2010/))), 
                climate pledges have forced leaders to steer away towards renewables 
                ([Bundesregierung, 2022](https://www.bundesregierung.de/breg-en/issues/climate-action/government-climate-policy-1779414)). 

                While contruction of onshore wind parks is repeatedly halted by local opposition ([DW, 2022](https://p.dw.com/p/4K361)),
                installed photovoltaic (PV) capacity has steadily and rapidly grown in the recent years
                ([Bundesnetzagentur, 2022](https://www.smard.de/home/marktdaten)). One of its great benefits is the much lower footprint
                of the panels compared to wind parks. This allows PV to be installed in places with little other use, 
                most prominently on house roofs. Several German states have in fact made PV mandatory on new buildings 
                ([Imolauer, 2022](https://www.roedl.com/insights/renewable-energy/2021/august/pv-obligation-germany-federal-states)). 

                Looking for further opportunities, the southern-German city of Tübingen found the space inside one of their highway ramps
                to be highly suited for PV use
                ([swt, 2022](https://www.swtue.de/energie/strom/erneuerbare-energien/bautagebuecher/solarpark-lustnauer-ohren.html)).
                After several years of legal hassle, the site is now the city's largest source of green energy.

                Its great benefit - similar to house roofs - is that there is little alternative use cases for the area.
                Encircled by traffic and pollution, the city and us authors believe that PV energy production is an ingenious idea
                to repurpose these human-made wastelands. Germany features a large network of divided laneways in its
                Autobahn and Bundesstraße road systems. The many connections to local roads may serve as excellent opportunities
                for PV energy production in the future.
            '''),
            dbc.Card([
                dbc.CardImg(src = BB_ohren, top=True),
                dbc.CardBody(
                    html.P("Beispielbild der potenziellen Flächen.", className="card-text")),
            ],style={"height": "70vh", "width": "70vh", 'padding': '10vh 5vh'},),
            html.Hr()
        ]),
        
        # Aufbau
        dbc.Row([
            html.H3('Aufbau', id="aufbau")
        ]),
        dbc.Row([
            dcc.Markdown('''
                Our project seeks to provide a comprehensive overview on the potential in equiping highway turnoffs across Germany with photovoltaics.
                Using state-of-the-art GIS software and Machine Learning techniques we identify turnoffs and evaluate their suitability.
                Beginning with a limited scope on the German state of Brandenburg, we plan to extend our models to cover the entire nation's road network.

                Core elements of the research project include:
                    - [x] Identification of highway turnoffs
                    - [ ] Evaluation of existing build-up and vegetation using satellite imagery and a Deep Neural Network
                    - [ ] Further economic considerations
                        - [ ] Proximity to power grid
                        - [ ] Terrain suitability
                        - [ ] Macro-level wheather considerations based on sunlight
                        - [ ] Optimal angle of construction and safety of passing cars
                    - [ ] Presentation of findings in a Dashboard

                Economic suitability of a potential spot is measured with the following economic model:
            '''),
            dbc.Card([
                html.Img(src = economic_model_img),
                dbc.CardBody(
                    html.P("Structure of the economic model.", className="card-text")),
            ],style={"height": "70vh", "width": "70vh", 'padding': '10vh 5vh'},),
            html.Hr()
        ]),
        
        # Sunburst Plot
        dbc.Row([
            html.H3('Überblick Landkreise', id = "ueberblick")
        ]),
        # Sunburst Plot
        dbc.Row([dbc.Col([
            dcc.Graph(figure = Ohren_sunburst, responsive=True, style = {'height': '100vh', 'width': '100vh'})
            ], class_name = "d-flex justify-content-center")
        ])

    ])

], style=CONTENT_STYLE)

keine Matrikelnummern
Namen Yvette ändern
alle Interviews aufnehmen
Piot Projket deutschalnd

In [11]:
app.layout = html.Div([sidebar, content])

# run app
if __name__ == '__main__':
    app.run_server()

Dash is running on http://127.0.0.1:8050/

 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:8050
Press CTRL+C to quit
127.0.0.1 - - [14/Feb/2023 15:33:24] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [14/Feb/2023 15:33:25] "GET /_dash-layout HTTP/1.1" 200 -
127.0.0.1 - - [14/Feb/2023 15:33:25] "GET /_dash-dependencies HTTP/1.1" 200 -
127.0.0.1 - - [14/Feb/2023 15:33:25] "GET /_dash-component-suites/dash/dcc/async-markdown.js HTTP/1.1" 304 -
127.0.0.1 - - [14/Feb/2023 15:33:25] "GET /_dash-component-suites/dash/dcc/async-graph.js HTTP/1.1" 304 -
127.0.0.1 - - [14/Feb/2023 15:33:25] "GET /_dash-component-suites/dash/dcc/async-plotlyjs.js HTTP/1.1" 304 -
127.0.0.1 - - [14/Feb/2023 15:33:25] "GET /_dash-component-suites/dash/dcc/async-highlight.js HTTP/1.1" 304 -


# Test Bereich